- 在本节中，我们将微调在 Penn-Fudan 数据库中对行人检测和分割的已预先训练的 Mask R-CNN 模型。它包含170个图像和345个行人实例，我们 将用它来说明如何在 torchvision 中使用新功能，以便在自定义数据集上训练实例分割模型。

# 1. 定义数据集
- 对于训练对象检测的引用脚本，实例分割和人员关键点检测要求能够轻松支持添加新的自定义数据。数据集应该从标准的类`torch.utils.data.Dataset` 继承而来，并实现`_len_`和`_getitem_`
- 我们要求的唯一特性是数据集的`__getitem__`应该返回： 
    - 图像：PIL图像大小(H,W) 
    - 目标：包含以下字段的字典
    - <1> `boxes(FloatTensor[N,4])`：N边框（bounding boxes）坐标的格式[x0,x1,y0,y1]，取值范围是0到W,0到H。
    - <2> `labels(Int64Tensor[N])`：每个边框的标签。
    - <3> `image_id(Int64Tensor[1])`：图像识别器，它应该在数据集中的所有图像中是唯一的，并在评估期间使用。
    - <4> `area(Tensor[N])`：边框的面积，在使用COCO指标进行评估时使用此项来分隔小、中和大框之间的度量标准得分。
    - <5> `iscrowed(UInt8Tensor[N,H,W])`：在评估期间属性设置为iscrowed=True的实例会被忽略。
    - <6> (可选)`masks(UInt8Tesor[N,H,W])`：每个对象的分段掩码。
    - <7> (可选)`keypoints (FloatTensor[N, K, 3])`：对于N个对象中的每一个，它包含[x，y，visibility]格式的K个关键点，用 于定义对象。`visibility = 0`表示关键点不可见。请注意，对于数据扩充，翻转关键点的概念取决于数据表示，您应该调整 reference/detection/transforms.py 以用于新的关键点表示。
- 如果你的模型返回上述方法，它们将使其适用于培训和评估，并将使用 pycocotools 的评估脚本。
- 此外，如果要在训练期间使用宽高比分组（以便每个批次仅包含具有相似宽高比的图像），则建议还实现`get_height_and_width`方法， 该方法返回图像的高度和宽度。如果未提供此方法，我们将通过`__getitem__`查询数据集的所有元素，这会将图像加载到内存中，但比提供自定义方法时要慢。

# 2. 为PennFudan编写自定义数据集
## 2.1 下载数据集
下载数据集和解压代码改为dsw使用!表示执行终端命令行程序

In [2]:
# download the Penn-Fudan dataset
!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip .   
# extract it in the current folder
!unzip -o -q PennFudanPed.zip
print("数据集下载完成，在DSW的download目录下")

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


数据集下载完成，在DSW的download目录下


'unzip' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


如果在windows中可以试着直接去网站上下载，把网站
https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
直接输到浏览器里头就能直接下载了


- 下面是一个图像以及其分割掩膜的例子：

<img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1614170561699-79868079-495a-4eb2-b81e-e1f7f5a5ab4e.png"/>
<img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1614170579441-fd3fe008-5455-44da-b1a9-018bd370e14f.png"/>

- 因此每个图像具有相应的分割掩膜，其中每个颜色对应于不同的实例。让我们为这个数据集写一个`torch.utils.data.Dataset`类。

## 2.2 为数据集编写类

In [24]:
import os
import numpy as np
import torch  # torch版本1.7.1(实测结果表明，torch版本影响不大)
from PIL import Image


class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # 下载所有图像文件，为其排序
        # 确保它们对齐
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # 请注意我们还没有将mask转换为RGB,
        # 因为每种颜色对应一个不同的实例
        # 0是背景
        mask = Image.open(mask_path)
        # 将PIL图像转换为numpy数组
        mask = np.array(mask)
        # 实例被编码为不同的颜色
        # 对于一维数组或者列表，np.unique() 函数 去除其中重复的元素 ，并按元素 由小到大 返回一个新的无元素重复的元组或者列表。
        obj_ids = np.unique(mask)
        # 第一个id是背景，所以删除它
        obj_ids = obj_ids[1:]

        # 将颜色编码的mask分成一组
        # 二进制格式
        masks = mask == obj_ids[:, None, None]

        # 获取每个mask的边界框坐标
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # 将所有转换为torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 这里仅有一个类
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 假设所有实例都不是人群
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        #当引入了预处理手段时，需要将处理完的数据进行进一步的转换
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

# 3. 定义模型
- 现在我们需要定义一个可以上述数据集执行预测的模型。在本节中，我们将使用 [Mask R-CNN](https://arxiv.org/abs/1703.06870)， 它基于 [Faster R-CNN](https://arxiv.org/abs/1506.01497)。Faster R-CNN 是一种模型，可以预测图像中潜在对象的边界框和类别得分。

<img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1614170660258-a4d0df39-059c-4955-b994-0b19d9abe429.png"/>

- Mask R-CNN 在 Faster R-CNN 中添加了一个额外的分支，它还预测每个实例的分割蒙版。

<img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1614170709652-fbc29488-3a9e-478a-989e-ebed8bae8a03.png"/>

- 有两种常见情况可能需要修改`torchvision modelzoo`中的一个可用模型。第一个是我们想要从预先训练的模型开始，然后微调最后一层。 另一种是当我们想要用不同的模型替换模型的主干时（例如，用于更快的预测）。
- 下面是对这两种情况的处理。 
    - 1 微调已经预训练的模型 让我们假设你想从一个在COCO上已预先训练过的模型开始，并希望为你的特定类进行微调。这是一种可行的方法：

In [2]:
# 安装包后需要加上提示⽤户重启kernel才能⽣效的语句
!pip install torchvision==0.8.2 --user

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


Windows用户就别瞅上头了，没法用的。乖乖打开cmd吧

In [7]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# 在COCO上加载经过预训练的预训练模型
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# 将分类器替换为具有用户定义的 num_classes的新分类器
num_classes = 2  # 1 class (person) + background
# 获取分类器的输入参数的数量
in_features = model.roi_heads.box_predictor.cls_score.in_features
# 用新的头部替换预先训练好的头部
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

- 2 修改模型以添加不同的主干

In [8]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# 加载预先训练的模型进行分类和返回
# 只有功能
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN需要知道骨干网中的输出通道数量。对于mobilenet_v2，它是1280，所以我们需要在这里添加它
backbone.out_channels = 1280

# 我们让RPN在每个空间位置生成5 x 3个锚点
# 具有5种不同的大小和3种不同的宽高比。 
# 我们有一个元组[元组[int]]
# 因为每个特征映射可能具有不同的大小和宽高比
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# 定义一下我们将用于执行感兴趣区域裁剪的特征映射，以及重新缩放后裁剪的大小。 
# 如果您的主干返回Tensor，则featmap_names应为[0]。 
# 更一般地，主干应该返回OrderedDict [Tensor]
# 并且在featmap_names中，您可以选择要使用的功能映射。
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# 将这些pieces放在FasterRCNN模型中
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

## 3.1 PennFudan数据集的实例分割模型
- 在我们的例子中，我们希望从预先训练的模型中进行微调，因为我们的数据集非常小，所以我们将遵循上述第一种情况。
- 这里我们还要计算实例分割掩膜，因此我们将使用 Mask R-CNN：

In [25]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # 加载在COCO上预训练的预训练的实例分割模型
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # 获取分类器的输入特征数
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 用新的头部替换预先训练好的头部
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 现在获取掩膜分类器的输入特征数
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # 并用新的掩膜预测器替换掩膜预测器
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    return model

In [47]:
models = get_model_instance_segmentation(num_classes)
print(models)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

- 就是这样，这将使模型准备好在您的自定义数据集上进行训练和评估。

# 4. 整合
- 在`references/detection/`中，我们有许多辅助函数来简化训练和评估检测模型。在这里，我们将使用 `references/detection/engine.py`，`references/detection/utils.py`和`references/detection/transforms.py`。 只需将它们复制到您的文件夹并在此处使用它们。

In [13]:
# !git clone https://github.com/pytorch/vision.git
!cp ./data/vision/references/detection/utils.py ./
!cp ./data/vision/references/detection/transforms.py ./
!cp ./data/vision/references/detection/coco_eval.py ./
!cp ./data/vision/references/detection/engine.py ./
!cp ./data/vision/references/detection/coco_utils.py ./

'cp' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'cp' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'cp' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'cp' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'cp' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


## 4.1 为数据扩充/转换编写辅助函数：

In [48]:
import transforms as T
import torchvision 

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    


    return T.Compose(transforms)

## 4.2 编写执行训练和验证的主要功能

In [17]:
# 安装包后需要加上提示⽤户重启kernel才能⽣效的语句
!pip install pycocotools --user

     --------------------------------------- 85.3/85.3 kB 47.5 kB/s eta 0:00:00


其实本质上，只有数据集部分和模型部分是存在不同的，所以只看这两部分与相关的一点东西就好了。

In [51]:
from engine import train_one_epoch, evaluate
import utils



# 在GPU上训练，若无GPU，可选择在CPU上训练
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 我们的数据集只有两个类 - 背景和人
num_classes = 2
# 使用我们的数据集和定义的转换
dataset = PennFudanDataset('./data/PennFudanPed/PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('./data/PennFudanPed/PennFudanPed', get_transform(train=False))

# 在训练和测试集中拆分数据集
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# 定义训练和验证数据加载器
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=0,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)
# 使用我们的辅助函数获取模型
model = get_model_instance_segmentation(num_classes)

# 将我们的模型迁移到合适的设备
model.to(device)

# 构造一个优化器
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.05)
# 和学习率调度程序
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# 训练2个epochs
num_epochs = 2

for epoch in range(num_epochs):
    # 训练一个epoch，每10次迭代打印一次
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # 更新学习速率
    lr_scheduler.step()
    # 在测试集上评价
    evaluate(model, data_loader_test, device=device)

print("That's it!")

TypeError: Expected input images to be of floating type (in range [0, 1]), but found type torch.uint8 instead

实测证明，torch版本还有有影响的


# 练习题
选择题：
1. 我们一般通过哪些参数来生成一组锚框  
    a. 锚框左上角xy坐标和右下角xy坐标  
    b. 锚框中心像素xy坐标和锚框长宽  
    c. 锚框中心像素xy坐标、锚框大小和宽高比  
    d. 锚框左上角xy坐标和锚框长宽  
2. 关于IoU说法错误的是  
    a. IoU是两个边界框相交面积与相并面积之比  
    b. IoU是两个边界框相减面积与相加面积之比  
    c. IoU的取值范围在0和1之间  
    d. IoU为0代表两个边界框无重合像素  
3. 在标注训练集的锚框时，下列说法正确的是  
    a. 在猫狗识别中，所有锚框被分为猫或狗两类  
    b. 每个锚框需要被标注所含目标类别和偏移量两个标签  
    c. 偏移量用中心点x坐标距离和中心点y坐标距离两个维度表示  
    d. 每个类别IoU最高的锚框标注为正类，其余锚框均为背景类  
4. 关于非极大值抑制说法错误的是  
    a. 用于解决边界框预测时同一个目标上可能会输出较多相似的预测边界框的问题  
    b. 预测边界框按置信度轮流作为基准  
    c. 将所有与基准预测边界框的交并比大于某阈值的非基准预测边界框移除  
    d. 将所有与基准预测边界框的交并比小于某阈值的非基准预测边界框移除  
    
答案：
1. c
2. b
3. b
4. d